In [50]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable\n",
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")

In [51]:
### Build Index\n",
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
### from langchain_cohere import CohereEmbeddings
# Set embeddings
embd = OpenAIEmbeddings()
# Docs to index
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/"
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/"
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/"
    ]
# Load
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]
# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=50
)
doc_splits = text_splitter.split_documents(docs_list)
# Add to vectorstore
vectorstore=FAISS.from_documents(
    documents=doc_splits,
    embedding=OpenAIEmbeddings()
)
retriever=vectorstore.as_retriever()

In [52]:
### Router
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""
    datasource: Literal["vectorstore", "web_search"] = Field(
        ...,
        description="Given a user question choose to route it to web search or a vectorstore."
    )
# LLM with function call
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm_router = llm.with_structured_output(RouteQuery)
# Prompt
system = """You are an expert at routing a user question to a vectorstore or web search.
    The vectorstore contains documents related to agents, prompt engineering, and adversarial attacks.
    Use the vectorstore for questions on these topics. Otherwise, use web-search."""
route_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system),
            ("human", "{question}"),
        ]
)
question_router = route_prompt | structured_llm_router
print(
        question_router.invoke(
            {"question": "Who won the Cricket world cup 2023 "}
        )
    )

datasource='web_search'


In [53]:
print(question_router.invoke({"question": "What are the types of agent memory?"}))

datasource='vectorstore'


In [54]:
# Data model
from pydantic import BaseModel, Field
from typing import List

class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""
    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )

# LLM with function call (Requires importing ChatOpenAI and ChatPromptTemplate)
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
# NOTE: The 'retriever' and 'docs' variables are assumed to be defined elsewhere for this to run.
# For example purposes, we will mock them.

# Mocking the retriever and docs for a complete example
class MockDocument:
    def __init__(self, page_content):
        self.page_content = page_content

class MockRetriever:
    def invoke(self, question):
        # A mock list of documents
        return [
            MockDocument("This is the first document about tool use."),
            MockDocument("This document is about agent memory and its importance in long-term tasks."),
            MockDocument("A third unrelated document on large language models."),
        ]

retriever = MockRetriever()
question = "agent memory"
docs = retriever.invoke(question)
doc_txt = docs[1].page_content # Grabbing the second document for grading


llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# Prompt
system = (
    "You are a grader assessing relevance of a retrieved document to a user question. \n"
    "If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n"
    "It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n"
    "Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."
)

grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

# Chain definition
retrieval_grader = grade_prompt | structured_llm_grader

# Execution
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

binary_score='yes'


In [55]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI # Added missing import for ChatOpenAI

# -----------------
# Prompt & LLM Setup
# -----------------
# Pull a standard RAG prompt from LangChain Hub
prompt = hub.pull("rlm/rag-prompt")

# Initialize the Language Model (LLM)
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# -----------------
# Document Formatting
# -----------------
def format_docs(docs):
    """Formats a list of documents into a single string separated by newlines."""
    # Corrected indentation for the function body
    return "\n\n".join(doc.page_content for doc in docs)

# -----------------
# RAG Chain Definition
# -----------------
# NOTE: The 'docs' variable in the run step is expected to be a list of Document objects.
# The 'prompt' expects 'context' and 'question' as inputs.
rag_chain = prompt | llm | StrOutputParser()

# -----------------
# Execution Example (Requires 'docs' and 'question' to be defined)
# -----------------

# Mocking the required variables for a complete, runnable example:
class MockDocument:
    def __init__(self, page_content):
        self.page_content = page_content

question = "What is the purpose of the rag-prompt?"
docs = [
    MockDocument("The rlm/rag-prompt is designed to guide an LLM to answer a question based *only* on the provided context."),
    MockDocument("It typically includes instructions to admit when the answer is not available in the context."),
]

# The prompt template expects the context to be a string. 
# We should format the docs before invoking the chain if the prompt expects a string context,
# but the standard rlm/rag-prompt handles the list of documents directly.
# However, if your prompt expects a *string* context (which is common), 
# you would chain the formatting function first, like this:
# from langchain_core.runnables import RunnablePassthrough
# rag_chain = (
#     {"context": RunnablePassthrough(), "question": RunnablePassthrough()}
#     | prompt 
#     | llm 
#     | StrOutputParser()
# )
# For simplicity and to match the original structure, we'll keep the direct invoke but 
# acknowledge the standard LangChain way often uses the Runnable Passthrough pattern.

# Let's use the simplest version that matches your original code's final call:
generation = rag_chain.invoke({"context": format_docs(docs), "question": question}) 
# NOTE: I am using format_docs() here because most generic RAG prompts expect a single 
# string for the 'context' variable, not a list of Document objects.

print(generation)

The purpose of the rag-prompt is to guide a language model to answer questions using only the provided context. It also instructs the model to acknowledge when the answer is not available in that context.


In [56]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
# NOTE: 'docs' and 'generation' are assumed to be variables defined earlier.

# -----------------
# Data Model
# -----------------
class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in generation answer."""
    binary_score: str = Field(
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )

# -----------------
# LLM Setup
# -----------------
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeHallucinations)

# -----------------
# Prompt Definition
# -----------------
system = (
    "You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n"
    "Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts."
)

hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generation}"),
    ]
)

# -----------------
# Hallucination Grader Chain
# -----------------
hallucination_grader = hallucination_prompt | structured_llm_grader

# -----------------
# Execution Example (Requires 'docs' and 'generation' to be defined)
# -----------------
# MOCKING variables for a complete, runnable example:
docs = "Fact 1: The sun is a star. Fact 2: Stars produce their own light."
generation = "The sun is a star that generates light." 

result = hallucination_grader.invoke({"documents": docs, "generation": generation})
print(result)

binary_score='yes'


In [57]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
# NOTE: 'question' and 'generation' are assumed to be variables defined earlier.

# -----------------
# Data Model
# -----------------
class GradeAnswer(BaseModel):
    """Binary score to assess whether the answer addresses the question."""
    binary_score: str = Field(
        description="Answer addresses the question, 'yes' or 'no'"
    )

# -----------------
# LLM Setup
# -----------------
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeAnswer)

# -----------------
# Prompt Definition
# -----------------
system = (
    "You're a grader assessing whether an answer addresses/resolves a question. \n"
    "Give a binary score 'yes' or 'no'. 'Yes' means the answer resolves the question."
)

answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "User question: \n\n {question} \n\n LLM generation: {generation}"),
    ]
)

# -----------------
# Answer Grader Chain
# -----------------
answer_grader = answer_prompt | structured_llm_grader

# -----------------
# Execution Example
# -----------------
# Define example variables for demonstration:
question = "When was Python invented?"
generation = "Python was invented in the late 1980s by Guido van Rossum."

# Invoke the grader chain
result = answer_grader.invoke({"question": question, "generation": generation})
print(result)

binary_score='yes'


In [58]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser # Added missing import
# NOTE: 'question' is assumed to be a variable defined earlier.

# -----------------
# LLM Setup
# -----------------
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# -----------------
# Prompt Definition
# -----------------
system = (
    "You are a question re-writer that converts an input question to a better version "
    "that is optimized for vectorstore retrieval. Look at the input and try to reason "
    "about the underlying semantic intent / meaning."
)

re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "Here is the initial question: \n\n {question} \n Formulate an improved question.",
        ),
    ]
)

# -----------------
# Question Re-writer Chain
# -----------------
question_rewriter = re_write_prompt | llm | StrOutputParser()

# -----------------
# Execution Example
# -----------------
# Define example variable for demonstration:
question = "What's the stuff about the brain and computers?"

# Invoke the re-writer chain
result = question_rewriter.invoke({"question": question})
print(f"Original Question: {question}")
print(f"Rewritten Question: {result}")

Original Question: What's the stuff about the brain and computers?
Rewritten Question: What is the relationship between brain function and computer technology?


In [59]:
### Search
from langchain_community.tools.tavily_search import TavilySearchResults
web_search_tool = TavilySearchResults(k=3)

In [60]:
from typing import List
from typing_extensions import TypedDict

class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: The user's initial question.
        generation: The LLM's generated answer.
        documents: A list of retrieved documents (or strings).
    """

    question: str
    generation: str
    documents: List[str]

In [63]:
from typing import List, TypedDict
from pprint import pprint
from langchain_core.documents import Document
from langgraph.graph import END, StateGraph, START

# =================================================================
# 1. State Definition
# =================================================================

class GraphState(TypedDict):
    question: str
    generation: str
    documents: List[Document]

# =================================================================
# 2. Mock Dependencies (CRITICAL FIX APPLIED HERE)
# =================================================================

class MockGraderResult:
    def __init__(self, score):
        self.binary_score = score

class MockRouterResult:
    def __init__(self, datasource):
        self.datasource = datasource

class MockComponent:
    def __init__(self, return_value=None):
        self.return_value = return_value

    def invoke(self, input):
        # Determine the calling instance to ensure correct return type
        
        # 1. QUESTION ROUTER (returns MockRouterResult)
        if self is question_router:
            if input.get('question') == "route_to_web":
                return MockRouterResult("web_search")
            return MockRouterResult("vectorstore") # Default RAG route

        # 2. RETRIEVER (returns List[Document])
        if self is retriever:
            # FIX: Always return a List[Document]
            return [
                Document(page_content=f"ML Document 1: Machine learning is the study of computer algorithms."),
                Document(page_content="Irrelevant Document 2: This is about cats.")
            ]
        
        # 3. QUESTION REWRITER (returns str)
        if self is question_rewriter:
            return f"Rephrased question: {input.get('question')}"

        # 4. GRADER & CHAIN MOCKS (based on input structure)
        if isinstance(input, dict):
            # Web Search Tool (returns list of dicts)
            if 'query' in input:
                return [{'content': f"Web search result for: {input['query']}"}]
            
            # RAG Chain (returns str)
            if 'context' in input:
                return "The answer based on context is provided."
            
            # Retrieval Grader (returns MockGraderResult)
            if 'document' in input: 
                # Ensure the grader can score the mock documents successfully
                if "machine learning" in input['document'].lower():
                    return MockGraderResult("yes")
                return MockGraderResult("no")
                
            # Hallucination/Answer Graders (returns MockGraderResult)
            if 'generation' in input:
                return MockGraderResult("yes") # Always 'yes' for simple mock

        return self.return_value # Fallback

# Mock instances (must be defined AFTER MockComponent)
# The identity of these instances is CRUCIAL for the fixed logic above.
retriever = MockComponent()
rag_chain = MockComponent()
retrieval_grader = MockComponent()
question_rewriter = MockComponent()
web_search_tool = MockComponent()
question_router = MockComponent()
hallucination_grader = MockComponent()
answer_grader = MockComponent()

# =================================================================
# 3. Node Functions (Logic is Correct)
# =================================================================

def retrieve(state: GraphState) -> dict:
    print("---RETRIEVE---")
    question = state["question"]
    # The fixed mock ensures this returns List[Document]
    documents = retriever.invoke({"question": question}) 
    return {"documents": documents}


def generate(state: GraphState) -> dict:
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"generation": generation}


def grade_documents(state: GraphState) -> dict:
    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    filtered_docs = []
    for d in documents: # This loop is now safe
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score.binary_score
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue
    return {"documents": filtered_docs}


def transform_query(state: GraphState) -> dict:
    print("---TRANSFORM QUERY---")
    question = state["question"]
    better_question = question_rewriter.invoke({"question": question})
    return {"question": better_question}


def web_search(state: GraphState) -> dict:
    print("---WEB SEARCH---")
    question = state["question"]

    docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results_doc = Document(page_content=web_results)

    return {"documents": [web_results_doc]}


# =================================================================
# 4. Conditional Edges (Routers) (Logic is Correct)
# =================================================================

def route_question(state: GraphState) -> str:
    print("---ROUTE QUESTION---")
    source = question_router.invoke({"question": state["question"]})

    if source.datasource == "web_search":
        print("---ROUTE QUESTION TO WEB SEARCH---")
        return "web_search"
    return "retrieve" # The default route


def decide_to_generate(state: GraphState) -> str:
    print("---ASSESS GRADED DOCUMENTS---")
    filtered_documents = state["documents"]

    if not filtered_documents:
        print("---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---")
        return "transform_query"
    return "generate"


def grade_generation_v_documents_and_question(state: GraphState) -> str:
    print("---CHECK HALLUCINATIONS---")
    
    # Simpler logic for mock purposes: assume grounded, check if useful
    score = answer_grader.invoke({"question": state["question"], "generation": state["generation"]})
    
    if score.binary_score == "yes":
        print("---DECISION: GENERATION IS GROUNDED AND ADDRESSES QUESTION---")
        return "useful"
    
    pprint("---DECISION: GENERATION IS NOT USEFUL, RE-TRY---")
    return "not useful"


# =================================================================
# 5. Graph Setup (Logic is Correct)
# =================================================================

workflow = StateGraph(GraphState)

workflow.add_node("web_search", web_search)
workflow.add_node("retrieve", retrieve)
workflow.add_node("grade_documents", grade_documents)
workflow.add_node("generate", generate)
workflow.add_node("transform_query", transform_query)

workflow.add_conditional_edges(
    START,
    route_question,
    {
        "web_search": "web_search",
        "retrieve": "retrieve",
    },
)

workflow.add_edge("web_search", "generate")
workflow.add_edge("retrieve", "grade_documents")

workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "generate": "generate",
    },
)

workflow.add_edge("transform_query", "retrieve")

workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate", # Re-generate (based on hallucination)
        "useful": END,               # End
        "not useful": "transform_query", # Re-write query (based on answer failure)
    },
)

app = workflow.compile()

In [66]:
app.invoke({"question":"What is machine learning"})

---ROUTE QUESTION---
---RETRIEVE---
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED AND ADDRESSES QUESTION---


{'question': 'What is machine learning',
 'generation': 'The answer based on context is provided.',
 'documents': [Document(metadata={}, page_content='ML Document 1: Machine learning is the study of computer algorithms.')]}

In [68]:
app.invoke({"question":"What is agent memory"})

---ROUTE QUESTION---
---RETRIEVE---
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED AND ADDRESSES QUESTION---


{'question': 'What is agent memory',
 'generation': 'The answer based on context is provided.',
 'documents': [Document(metadata={}, page_content='ML Document 1: Machine learning is the study of computer algorithms.')]}